# Init

In [4]:
import requests
import json
from datetime import datetime

def test_proxy_detailed():
    proxies = {
        'http': 'http://127.0.0.1:7890',
        'https': 'http://127.0.0.1:7890'
    }
    
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 开始测试代理连接...")
    
    # 测试 Google 连接
    try:
        start = datetime.now()
        r = requests.get('https://www.google.com', proxies=proxies, timeout=5)
        elapsed = (datetime.now() - start).total_seconds()
        print(f"✅ Google 连接成功! 状态码: {r.status_code}, 响应时间: {elapsed:.2f}秒")
    except Exception as e:
        print(f"❌ Google 连接失败: {e}")
    
    # 获取 IP 信息
    try:
        ip_info = requests.get('https://ipinfo.io/json', proxies=proxies, timeout=5).json()
        print(f"📍 当前 IP 信息: {json.dumps(ip_info, indent=2, ensure_ascii=False)}")
    except Exception as e:
        print(f"❌ 获取 IP 信息失败: {e}")
    
    # 测试延迟
    try:
        r = requests.get('https://www.cloudflare.com/cdn-cgi/trace', proxies=proxies, timeout=5)
        cf_info = dict(line.split('=', 1) for line in r.text.splitlines() if '=' in line)
        print(f"🌐 Cloudflare 信息: 位置={cf_info.get('loc', '未知')}, warp={cf_info.get('warp', '未知')}")
    except Exception as e:
        print(f"❌ Cloudflare 测试失败: {e}")

# 执行测试
test_proxy_detailed()

[2025-07-18 16:09:18] 开始测试代理连接...
✅ Google 连接成功! 状态码: 200, 响应时间: 1.20秒
📍 当前 IP 信息: {
  "ip": "188.253.121.162",
  "city": "Singapore",
  "region": "Singapore",
  "country": "SG",
  "loc": "1.2897,103.8501",
  "org": "AS38136 Akari Networks",
  "postal": "018989",
  "timezone": "Asia/Singapore",
  "readme": "https://ipinfo.io/missingauth"
}
🌐 Cloudflare 信息: 位置=SG, warp=off


In [ ]:
import geemap
import ee
import geopandas as gpd
import os
import numpy as np
from tqdm import tqdm
import yaml
from pathlib import Path

geemap.set_proxy(port='7890')
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [5]:
test = ee.List(['HH', 'HV'])
test.get(0)

In [ ]:
import torch  
torch.cuda.empty_cache()

# GetPixels 采样点取不同时间影像值

In [4]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [10]:



# Region of interest.
coords = [
    -121.58626826832939,
    38.059141484827485,
]
region = ee.Geometry.Point(coords)

# Sentinel-2 median composite.
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(region)
              .filterDate('2020-04-01', '2020-09-01')
              .median())

# Make a projection to discover the scale in degrees.
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()

# Get scales out of the transform.
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Make a request object.
request = {
    'expression': image,
    'fileFormat': 'PNG',
    'bandIds': ['B4', 'B3', 'B2'],
    'grid': {
        'dimensions': {
            'width': 640,
            'height': 640
        },
        'affineTransform': {
            'scaleX': scale_x,
            'shearX': 0,
            'translateX': coords[0],
            'shearY': 0,
            'scaleY': scale_y,
            'translateY': coords[1]
        },
        'crsCode': proj['crs'],
    },
    'visualizationOptions': {'ranges': [{'min': 0, 'max': 3000}]},
}

image_png = ee.data.computePixels(request)

In [11]:
# Region of interest.
coords = [
    -121.58626826832939,
    38.059141484827485,
]
region = ee.Geometry.Point(coords)

# Get a Sentinel-2 image.
image = (ee.ImageCollection('COPERNICUS/S2')
  .filterBounds(region)
  .filterDate('2020-04-01', '2020-09-01')
  .sort('CLOUD_COVERAGE_ASSESSMENT')
  .first())
image_id = image.getInfo()['id']

# Make a projection to discover the scale in degrees.
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()

# Get scales out of the transform.
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Make a request object.
request = {
    'assetId': image_id,
    'fileFormat': 'PNG',
    'bandIds': ['B4', 'B3', 'B2'],
    'grid': {
        'dimensions': {
            'width': 640,
            'height': 640
        },
        'affineTransform': {
            'scaleX': scale_x,
            'shearX': 0,
            'translateX': coords[0],
            'shearY': 0,
            'scaleY': scale_y,
            'translateY': coords[1]
        },
        'crsCode': proj['crs'],
    },
    'visualizationOptions': {'ranges': [{'min': 0, 'max': 3000}]},
}

image_png = ee.data.getPixels(request)

In [ ]:
image_png

AttributeError: 'bytes' object has no attribute 'save'